# Competição DSA 2019/06
<h3>Objetivo: Prever Índice de Lealdade</h3>
<h4>Informações gerais disponíveis no <a href="https://www.kaggle.com/t/84f9cb851e9a428fa9fe54644df35f5c">Kaggle</a> </h4>
<h4>Parte 1: processo de ETL</h4>

In [1]:
# Importar as bibliotecas necessárias para este notebook
import pandas as pd
import numpy  as np
from datetime import datetime as dt
from scipy       import stats
from collections import OrderedDict
from sklearn.model_selection import KFold
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import mean_squared_error
from sklearn.linear_model    import LinearRegression

import warnings
warnings.filterwarnings("ignore")

# Configuração do comando head()
pd.set_option('display.max_columns', 50)

In [2]:
# importar os arquivos para dataframes:

# Contêm card_ids e informações sobre o próprio cartão - o primeiro mês em que o cartão estava ativo, etc
# O arquivo dataset_treino.csv também contém o target (índice de lealdade do cliente)
train_file        = pd.read_csv('dataset_treino.csv')
test_file         = pd.read_csv('dataset_teste.csv')
comerciantes_file = pd.read_csv('comerciantes.csv')
# Contêm informações sobre as transações de cada cartão:
historico_cartoes = pd.read_csv('transacoes_historicas.csv')  # contém até três meses de transações para cada cartão
historico_comerci = pd.read_csv('novas_transacoes_comerciantes.csv')

<h3>ETL do arquivo train_file</h3>

In [3]:
# Verificar presença de valores NAN
train_file.isna().sum()

first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64

In [4]:
# Dados estatísticos do data frame
train_file.describe()

,feature_1,feature_2,feature_3,target
count,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,-0.393636
std,1.186160,0.751362,0.495683,3.850500
min,1.000000,1.000000,0.000000,-33.219281
25%,2.000000,1.000000,0.000000,-0.883110
50%,3.000000,2.000000,1.000000,-0.023437
75%,4.000000,2.000000,1.000000,0.765453
max,5.000000,3.000000,1.000000,17.965068


In [5]:
# Variáveis feature são categóricas, verificar suas quantidades
print('feature_1:\n'); train_file.feature_1.value_counts()

feature_1:



3    73573
2    55797
5    40625
4    19885
1    12037
Name: feature_1, dtype: int64

In [6]:
print('feature_2:\n'); train_file.feature_2.value_counts()

feature_2:



1    89242
2    74839
3    37836
Name: feature_2, dtype: int64

In [7]:
print('feature_3:\n'); train_file.feature_3.value_counts()

feature_3:



1    114198
0     87719
Name: feature_3, dtype: int64

In [8]:
# Como feature_1 e feature_2 possuem mais de duas possibilidades, serão criadas variáveis dummie:
train_file = pd.get_dummies(train_file, columns = ['feature_1', 'feature_2'])

In [9]:
# Organizar a ordem das colunas:
train_file['feature_3_1_0'] = train_file['feature_3']
# Criar campo como tipo data
train_file['date_Card_Activation'] = pd.to_datetime(train_file['first_active_month'], format = "%Y-%m-%d")
train_file['alvo'] = train_file['target']
# Excluir campos desnecessários
train_file.drop(['feature_3', 'first_active_month', 'target'], axis = 1, inplace = True)

In [10]:
# Visualizar o resultado em train_file:
train_file.head(10)

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,date_Card_Activation,alvo
0,C_ID_92a2005557,0,0,0,0,1,0,1,0,1,2017-06-01,-0.820283
1,C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,2017-01-01,0.392913
2,C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,2016-08-01,0.688056
3,C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,2017-09-01,0.142495
4,C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,2017-11-01,-0.159749
5,C_ID_0894217f2f,0,0,0,1,0,0,1,0,0,2016-09-01,0.871585
6,C_ID_7e63323c00,0,0,1,0,0,0,1,0,1,2016-12-01,0.230129
7,C_ID_dfa21fc124,0,0,1,0,0,0,1,0,1,2017-09-01,2.135850
8,C_ID_fe0fdac8ea,0,1,0,0,0,1,0,0,0,2017-08-01,-0.065406
9,C_ID_bf62c0b49d,0,1,0,0,0,0,1,0,0,2016-08-01,0.300062


<h3>ETL do arquivo test_file</h3>

In [11]:
# Verificar presença de valores NAN
test_file.isna().sum()
# existe um valor NAN na variável first_active_month

first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [12]:
# Dados estatísticos do data frame
test_file.describe()

,feature_1,feature_2,feature_3
count,123623.00000,123623.000000,123623.000000
mean,3.10926,1.741796,0.564377
std,1.18911,0.749195,0.495840
min,1.00000,1.000000,0.000000
25%,2.00000,1.000000,0.000000
50%,3.00000,2.000000,1.000000
75%,4.00000,2.000000,1.000000
max,5.00000,3.000000,1.000000


In [13]:
# Variáveis feature são categóricas, verificar suas quantidades
print('feature_1:\n'); test_file.feature_1.value_counts()

feature_1:



3    44719
2    34115
5    25051
4    12332
1     7406
Name: feature_1, dtype: int64

In [14]:
print('feature_2:\n'); test_file.feature_2.value_counts()

feature_2:



1    54775
2    45993
3    22855
Name: feature_2, dtype: int64

In [15]:
print('feature_3:\n'); test_file.feature_3.value_counts()

feature_3:



1    69770
0    53853
Name: feature_3, dtype: int64

In [16]:
# As características de test_file são as mesmas do arquivo de train_file, o mesmo tratamento será dado
# Criar as variáveis dummie:
test_file = pd.get_dummies(test_file, columns = ['feature_1', 'feature_2'])
# Organizar a ordem das colunas:
test_file['feature_3_1_0'] = test_file['feature_3']
# Criar campo como tipo data
test_file['date_Card_Activation'] = pd.to_datetime(test_file['first_active_month'], format = "%Y-%m-%d")
# Excluir campos desnecessários
test_file.drop(['feature_3', 'first_active_month'], axis = 1, inplace = True)

In [17]:
# Resolver a variável NAN:
test_file['date_Card_Activation'].describe()

count                  123622
unique                     75
top       2017-09-01 00:00:00
freq                     8805
first     2011-11-01 00:00:00
last      2018-01-01 00:00:00
Name: date_Card_Activation, dtype: object

In [18]:
# Alterar o valor NAN para a mais recente
test_file['date_Card_Activation'].fillna(test_file.date_Card_Activation.max(), inplace = True); test_file.isna().sum()

card_id                 0
feature_1_1             0
feature_1_2             0
feature_1_3             0
feature_1_4             0
feature_1_5             0
feature_2_1             0
feature_2_2             0
feature_2_3             0
feature_3_1_0           0
date_Card_Activation    0
dtype: int64

In [19]:
# Visualizar test_file resultante:
test_file.head(10)

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,date_Card_Activation
0,C_ID_0ab67a22ab,0,0,1,0,0,0,0,1,1,2017-04-01
1,C_ID_130fd0cbdd,0,1,0,0,0,0,0,1,0,2017-01-01
2,C_ID_b709037bc5,0,0,0,0,1,1,0,0,1,2017-08-01
3,C_ID_d27d835a9f,0,1,0,0,0,1,0,0,0,2017-12-01
4,C_ID_2b5e3df5c2,0,0,0,0,1,1,0,0,1,2015-12-01
5,C_ID_5814b4f13c,0,0,0,0,1,1,0,0,1,2017-07-01
6,C_ID_a1b3c75277,1,0,0,0,0,0,1,0,0,2017-10-01
7,C_ID_f7cada36d3,0,0,1,0,0,1,0,0,1,2017-02-01
8,C_ID_9d2bc8dfc4,0,0,0,1,0,0,0,1,0,2017-07-01
9,C_ID_6d8dba8475,0,0,0,0,1,1,0,0,1,2016-03-01


In [20]:
# Comparar se as observações dos arquivos train_file e test_file possuem mesmo datatype
print(train_file.info())
print(test_file.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 12 columns):
card_id                 201917 non-null object
feature_1_1             201917 non-null uint8
feature_1_2             201917 non-null uint8
feature_1_3             201917 non-null uint8
feature_1_4             201917 non-null uint8
feature_1_5             201917 non-null uint8
feature_2_1             201917 non-null uint8
feature_2_2             201917 non-null uint8
feature_2_3             201917 non-null uint8
feature_3_1_0           201917 non-null int64
date_Card_Activation    201917 non-null datetime64[ns]
alvo                    201917 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), uint8(8)
memory usage: 7.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 11 columns):
card_id                 123623 non-null object
feature_1_1             123623 non-null uint8
feature_1_2             1

<h3>ETL do arquivo comerciantes_file</h3>

In [21]:
# Verificar presença de valores NAN
comerciantes_file.isna().sum()
# Existem muitos NAN a serem tratados

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [22]:
# Dados estatísticos do data frame
comerciantes_file.describe()
# verificada a presença de valores inf

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,city_id,state_id,category_2
count,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,334683.000000,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000,334696.000000,334696.000000,322809.000000
mean,31028.736143,423.131663,25.116404,0.011476,0.008103,13.832993,inf,2.994108,2.165079e+01,inf,5.947397,2.522771e+01,inf,11.599335,102.917926,11.860942,2.380002
std,31623.043426,252.898046,9.807371,1.098154,1.070497,2395.489999,NaN,0.095247,3.947108e+03,NaN,0.394936,5.251842e+03,NaN,1.520138,107.090673,6.176889,1.562661
min,1.000000,-1.000000,-1.000000,-0.057471,-0.057471,-82.130000,3.334953e-01,1.000000,-8.213000e+01,1.670447e-01,1.000000,-8.213000e+01,9.832954e-02,1.000000,-1.000000,-1.000000,1.000000
25%,3612.000000,222.000000,19.000000,-0.057471,-0.057471,0.880000,9.236499e-01,3.000000,8.500000e-01,9.022475e-01,6.000000,8.500000e-01,8.983333e-01,12.000000,-1.000000,9.000000,1.000000
50%,19900.000000,373.000000,27.000000,-0.057471,-0.057471,1.000000,1.016667e+00,3.000000,1.010000e+00,1.026961e+00,6.000000,1.020000e+00,1.043361e+00,12.000000,69.000000,9.000000,2.000000
75%,51707.250000,683.000000,33.000000,-0.047556,-0.047556,1.160000,1.146522e+00,3.000000,1.230000e+00,1.215575e+00,6.000000,1.290000e+00,1.266480e+00,12.000000,182.000000,16.000000,4.000000
max,112586.000000,891.000000,41.000000,183.735111,182.079322,851844.640000,inf,3.000000,1.513959e+06,inf,6.000000,2.567408e+06,inf,12.000000,347.000000,24.000000,5.000000


In [23]:
# Variáveis categóricas que referenciam região/setor:
# merchant_category_id, subsector_id, city_id, state_id; todas possuem valores -1
print(comerciantes_file[comerciantes_file['merchant_category_id'] < 1]['merchant_id'].count())
print(comerciantes_file[comerciantes_file['subsector_id'] < 1]['subsector_id'].count())
print(comerciantes_file[comerciantes_file['city_id'] < 1]['city_id'].count())
print(comerciantes_file[comerciantes_file['state_id'] < 1]['state_id'].count())

1
1
105184
11887


In [24]:
# city_id e state_id possuem muitos valores -1, já merchant_category_id e subsector_id possuem apenas uma observação
# com o mesmo id
# Pela pouca proporção dos valores NAN frente ao total de observações do dataframe, faz-se a opção pela remoção de 
# linhas com valores NAN
comerciantes_file = comerciantes_file.dropna()

In [25]:
# Com isso, somente city_id fica com valores -1; Opta-se por remover esta coluna
comerciantes_file.drop(['city_id'], axis=1, inplace = True)

In [26]:
# Variáveis categóricas que referenciam hierarquia:
# category_1, category_2, category_4, most_recent_sales_range, most_recent_purchases_range
print(comerciantes_file.category_1.value_counts())
print(comerciantes_file.category_2.value_counts())
print(comerciantes_file.category_4.value_counts())
print(comerciantes_file.most_recent_sales_range.value_counts())
print(comerciantes_file.most_recent_purchases_range.value_counts())

N    322802
Name: category_1, dtype: int64
1.0    160882
5.0     52922
3.0     51887
4.0     36450
2.0     20661
Name: category_2, dtype: int64
N    233744
Y     89058
Name: category_4, dtype: int64
E    172085
D    112849
C     32224
B      4732
A       912
Name: most_recent_sales_range, dtype: int64
E    168222
D    115430
C     33339
B      4864
A       947
Name: most_recent_purchases_range, dtype: int64


In [27]:
# Criar as variáveis dummie:
comerciantes_file = pd.get_dummies(comerciantes_file, columns = ['category_1', 'category_2', 'most_recent_sales_range', 'most_recent_purchases_range'])

In [28]:
# Em category_4, substituir Y por 0 e não por 1
comerciantes_file['category_4'] = comerciantes_file['category_4'].map({'N': 1, 'Y': 0})

In [29]:
# Ver datatypes do dataframe
comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322802 entries, 0 to 334695
Data columns (total 33 columns):
merchant_id                      322802 non-null object
merchant_group_id                322802 non-null int64
merchant_category_id             322802 non-null int64
subsector_id                     322802 non-null int64
numerical_1                      322802 non-null float64
numerical_2                      322802 non-null float64
avg_sales_lag3                   322802 non-null float64
avg_purchases_lag3               322802 non-null float64
active_months_lag3               322802 non-null int64
avg_sales_lag6                   322802 non-null float64
avg_purchases_lag6               322802 non-null float64
active_months_lag6               322802 non-null int64
avg_sales_lag12                  322802 non-null float64
avg_purchases_lag12              322802 non-null float64
active_months_lag12              322802 non-null int64
category_4                       322802 non-nul

In [30]:
# visualizar as primeiras linhas:
comerciantes_file.head(10)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,state_id,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,1,9,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,1,16,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,1,5,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
16,M_ID_c929bb59af,9514,2,20,-0.057471,-0.057471,69667.00,1.000000,3,69667.00,1.000000,3,69667.00,1.000000,3,1,20,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
17,M_ID_dd3ae3de10,30534,278,37,-0.057471,-0.057471,5180.00,4.000000,3,9830.00,4.166667,6,15826.00,5.600000,10,0,23,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
18,M_ID_dc671140d0,21536,278,37,-0.057471,-0.057471,7133.00,4.666667,3,17566.00,4.500000,6,19594.00,6.000000,9,0,5,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
19,M_ID_7551d55443,16010,298,10,-0.057471,-0.057471,109733.00,18.666667,3,118433.00,19.000000,6,217857.00,35.416667,12,1,18,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
20,M_ID_a8effe9197,577,497,27,-0.057471,-0.057471,3897.00,1.333333,3,2924.00,1.250000,4,2924.00,1.250000,4,1,16,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
21,M_ID_39643c7d93,22146,309,21,-0.057471,-0.057471,608433.00,5.666667,3,776683.00,7.000000,6,749804.00,6.000000,12,1,16,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
22,M_ID_07930b58ae,11559,705,33,-0.057471,-0.057471,148515.00,76.666667,3,245840.00,134.833333,6,272061.00,151.454545,11,1,19,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1


<h3>ETL do arquivo historico_cartoes</h3>

In [31]:
# Verificar presença de valores NAN
historico_cartoes.isna().sum()

authorized_flag             0
card_id                     0
city_id                     0
category_1                  0
installments                0
category_3                609
merchant_category_id        0
merchant_id               924
month_lag                   0
purchase_amount             0
purchase_date               0
category_2              14767
state_id                    0
subsector_id                0
dtype: int64

In [32]:
# Dados estatísticos do data frame
historico_cartoes.describe()

,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
count,199999.000000,199999.000000,199999.000000,199999.000000,199999.000000,185232.000000,199999.000000,199999.000000
mean,125.771759,0.495452,468.656738,-5.089705,-0.470251,2.150006,10.781014,27.631368
std,100.633496,0.927371,248.674052,3.712166,32.755567,1.518472,6.168744,9.492934
min,-1.000000,-1.000000,-1.000000,-13.000000,-0.746893,1.000000,-1.000000,-1.000000
25%,53.000000,0.000000,278.000000,-8.000000,-0.722445,1.000000,9.000000,19.000000
50%,88.000000,0.000000,437.000000,-5.000000,-0.694315,1.000000,9.000000,33.000000
75%,200.000000,1.000000,705.000000,-2.000000,-0.622488,3.000000,16.000000,34.000000
max,347.000000,12.000000,891.000000,0.000000,11269.666518,5.000000,24.000000,41.000000


In [33]:
# Observadas algumas inconsistências: valores negativos em todos os campos, exceto category_2
# Por isso, opta-se por excluir observações com valores NAN
historico_cartoes = historico_cartoes.dropna()

In [34]:
# Variável category_2 é categórica, verificar suas quantidades
print('category_2:\n'); historico_cartoes.category_2.value_counts()

category_2:



1.0    108423
3.0     27208
5.0     24957
4.0     16947
2.0      6270
Name: category_2, dtype: int64

In [35]:
# Variável category_3 é categórica, verificar suas quantidades
print('category_3:\n'); historico_cartoes.category_3.value_counts()

category_3:



A    116699
B     61865
C      5241
Name: category_3, dtype: int64

In [36]:
# Criar as variáveis dummie:
historico_cartoes = pd.get_dummies(historico_cartoes, columns = ['category_2', 'category_3'])

In [37]:
# visualizar quantidades em authorized_flag
print('authorized_flag:\n'); historico_cartoes.authorized_flag.value_counts()

authorized_flag:



Y    172945
N     10860
Name: authorized_flag, dtype: int64

In [38]:
# Em authorized_flag, substituir Y por 0 e não por 1
historico_cartoes['authorized_flag'] = historico_cartoes['authorized_flag'].map({'N': 1, 'Y': 0})

In [39]:
# Alterar ordem das colunas:
historico_cartoes['Unauthorized_card_purchase'] = historico_cartoes.authorized_flag
historico_cartoes.drop('authorized_flag', axis = 1, inplace = True)

In [40]:
# Variável category_1 é categórica, verificar suas quantidades
print('category_1:\n'); historico_cartoes.category_1.value_counts()

category_1:



N    183805
Name: category_1, dtype: int64

In [41]:
# Variável category_1 pode ser excluída, pois possui somente um valor
historico_cartoes.drop('category_1', axis = 1, inplace = True)

In [42]:
# Criar campo como tipo data
historico_cartoes['Purchase_transaction_date'] = pd.to_datetime(historico_cartoes['purchase_date'], format = "%Y-%m-%d")
# Exclusão do campo data anterior
historico_cartoes.drop('purchase_date', axis = 1, inplace=True)

In [43]:
# Visualizar historico_cartoes resultante:
historico_cartoes.head(10)

,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,Unauthorized_card_purchase,Purchase_transaction_date
0,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-7,-0.733128,16,16,1,0,0,0,0,1,0,0,0,2017-07-15 12:10:45
1,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-6,-0.720386,16,37,1,0,0,0,0,1,0,0,0,2017-08-09 22:04:29
2,C_ID_4e6213e9bc,88,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,16,34,1,0,0,0,0,1,0,0,0,2017-09-02 10:06:26
3,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-11,-0.722865,16,37,1,0,0,0,0,1,0,0,0,2017-03-10 01:14:19
4,C_ID_4e6213e9bc,333,0,80,M_ID_50af771f8d,0,-0.734887,9,37,1,0,0,0,0,1,0,0,0,2018-02-24 08:45:05
5,C_ID_4e6213e9bc,88,0,278,M_ID_5e8220e564,-11,-0.716855,16,37,1,0,0,0,0,1,0,0,0,2017-03-21 00:10:51
6,C_ID_4e6213e9bc,3,0,80,M_ID_9d41786a50,-3,-0.657049,16,37,1,0,0,0,0,1,0,0,0,2017-11-18 20:05:55
7,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-8,-0.737967,16,37,1,0,0,0,0,1,0,0,0,2017-06-01 22:02:56
8,C_ID_4e6213e9bc,88,0,80,M_ID_74ba14b5fc,-11,-0.715352,16,37,1,0,0,0,0,1,0,0,0,2017-03-16 15:41:22
9,C_ID_4e6213e9bc,88,0,683,M_ID_1449f22bfb,-9,-0.734135,16,34,1,0,0,0,0,1,0,0,0,2017-05-09 12:42:07


<h3>ETL do arquivo historico_comerci</h3>

In [44]:
# Verificar presença de valores NAN
historico_comerci.isna().sum()

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64

In [45]:
# Dados estatísticos do data frame
historico_comerci.describe()

,city_id,installments,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id
count,1.963031e+06,1.963031e+06,1.963031e+06,1.963031e+06,1.963031e+06,1.851286e+06,1.963031e+06,1.963031e+06
mean,1.343867e+02,6.829643e-01,4.309701e+02,1.476515e+00,-5.509690e-01,2.197841e+00,1.088067e+01,2.597624e+01
std,1.015152e+02,1.584069e+00,2.463385e+02,4.994483e-01,6.940043e-01,1.528125e+00,6.038542e+00,1.012908e+01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-7.468928e-01,1.000000e+00,-1.000000e+00,-1.000000e+00
25%,6.900000e+01,0.000000e+00,2.780000e+02,1.000000e+00,-7.166294e-01,1.000000e+00,9.000000e+00,1.900000e+01
50%,1.100000e+02,1.000000e+00,3.670000e+02,1.000000e+00,-6.748406e-01,1.000000e+00,9.000000e+00,2.900000e+01
75%,2.120000e+02,1.000000e+00,6.830000e+02,2.000000e+00,-5.816162e-01,3.000000e+00,1.500000e+01,3.400000e+01
max,3.470000e+02,9.990000e+02,8.910000e+02,2.000000e+00,2.631575e+02,5.000000e+00,2.400000e+01,4.100000e+01


In [46]:
# Observadas algumas inconsistências: valores negativos em todos os campos, exceto category_2
# Por isso, opta-se por excluir observações com valores NAN
historico_comerci = historico_comerci.dropna()

In [47]:
# Variável category_2 é categórica, verificar suas quantidades
print('category_2:\n'); historico_comerci.category_2.value_counts()

category_2:



1.0    1017454
3.0     273223
5.0     248406
4.0     173214
2.0      63849
Name: category_2, dtype: int64

In [48]:
# Variável category_3 é categórica, verificar suas quantidades
print('category_3:\n'); historico_comerci.category_3.value_counts()

category_3:



A    882804
B    771132
C    122210
Name: category_3, dtype: int64

In [49]:
# Criar as variáveis dummie:
historico_comerci = pd.get_dummies(historico_comerci, columns = ['category_2', 'category_3'])

In [50]:
# visualizar quantidades em authorized_flag
print('authorized_flag:\n'); historico_comerci.authorized_flag.value_counts()

authorized_flag:



Y    1776146
Name: authorized_flag, dtype: int64

In [51]:
# Em authorized_flag, substituir Y por 0 e não por 1
historico_comerci['authorized_flag'] = historico_comerci['authorized_flag'].map({'N': 1, 'Y': 0})

In [52]:
# Alterar ordem das colunas:
historico_comerci['Unauthorized_card_purchase'] = historico_comerci.authorized_flag
historico_comerci.drop('authorized_flag', axis = 1, inplace = True)

In [53]:
# Variável category_1 é categórica, verificar suas quantidades
print('category_1:\n'); historico_comerci.category_1.value_counts()

category_1:



N    1776146
Name: category_1, dtype: int64

In [54]:
# Variável category_1 pode ser excluída, pois possui somente um valor
historico_comerci.drop('category_1', axis = 1, inplace = True)

In [55]:
# Criar campo como tipo data
historico_comerci['Purchase_transaction_date'] = pd.to_datetime(historico_comerci['purchase_date'], format = "%Y-%m-%d")
# Exclusão do campo data anterior
historico_comerci.drop('purchase_date', axis = 1, inplace=True)

In [56]:
# Visualizar historico_cartoes resultante:
historico_comerci.head(10)

,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,Unauthorized_card_purchase,Purchase_transaction_date
0,C_ID_415bb3a509,107,1,307,M_ID_b0c793002c,1,-0.557574,9,19,1,0,0,0,0,0,1,0,0,2018-03-11 14:57:36
1,C_ID_415bb3a509,140,1,307,M_ID_88920c89e8,1,-0.569580,9,19,1,0,0,0,0,0,1,0,0,2018-03-19 18:53:37
2,C_ID_415bb3a509,330,1,507,M_ID_ad5237ef6b,2,-0.551037,9,14,1,0,0,0,0,0,1,0,0,2018-04-26 14:08:44
5,C_ID_ef55cf8d4b,231,1,367,M_ID_8874615e00,2,-0.633007,9,16,1,0,0,0,0,0,1,0,0,2018-04-02 12:53:28
6,C_ID_ef55cf8d4b,69,1,333,M_ID_6d061b5ddc,1,5.263697,9,10,1,0,0,0,0,0,1,0,0,2018-03-28 19:50:19
7,C_ID_ef55cf8d4b,231,1,307,M_ID_df1e022f41,2,-0.553787,9,19,1,0,0,0,0,0,1,0,0,2018-04-05 08:06:52
8,C_ID_ef55cf8d4b,69,1,278,M_ID_d15eae0468,2,-0.596643,9,37,1,0,0,0,0,0,1,0,0,2018-04-07 18:37:40
9,C_ID_ef55cf8d4b,69,1,437,M_ID_5f9bffd028,1,-0.607191,9,15,1,0,0,0,0,0,1,0,0,2018-03-17 18:10:41
11,C_ID_ef55cf8d4b,69,1,108,M_ID_e6f5213fbf,1,-0.605959,9,34,1,0,0,0,0,0,1,0,0,2018-03-11 12:53:41
12,C_ID_ef55cf8d4b,69,1,278,M_ID_aa97bc87f6,1,-0.634209,9,37,1,0,0,0,0,0,1,0,0,2018-03-14 14:07:43


<h3>Comparar historico_cartoes e historico_comerci visando concatená-los</h3>

In [57]:
# Comparar se as observações dos arquivos historico_cartoes e historico_comerci possuem mesmo datatype
print(historico_cartoes.info())
print(historico_comerci.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183805 entries, 0 to 199998
Data columns (total 19 columns):
card_id                       183805 non-null object
city_id                       183805 non-null int64
installments                  183805 non-null int64
merchant_category_id          183805 non-null int64
merchant_id                   183805 non-null object
month_lag                     183805 non-null int64
purchase_amount               183805 non-null float64
state_id                      183805 non-null int64
subsector_id                  183805 non-null int64
category_2_1.0                183805 non-null uint8
category_2_2.0                183805 non-null uint8
category_2_3.0                183805 non-null uint8
category_2_4.0                183805 non-null uint8
category_2_5.0                183805 non-null uint8
category_3_A                  183805 non-null uint8
category_3_B                  183805 non-null uint8
category_3_C                  183805 non-null uint8
U

In [58]:
# colunas city_id, category_1, installments, merchant_category_id, month_lag
historico_cartoes[['city_id', 'installments', 'merchant_category_id', 'month_lag']].describe()

,city_id,installments,merchant_category_id,month_lag
count,183805.000000,183805.000000,183805.000000,183805.000000
mean,134.900879,0.432431,456.212883,-5.164587
std,98.151157,0.767001,244.440731,3.715945
min,1.000000,0.000000,2.000000,-13.000000
25%,69.000000,0.000000,278.000000,-8.000000
50%,98.000000,0.000000,422.000000,-5.000000
75%,212.000000,1.000000,705.000000,-2.000000
max,347.000000,12.000000,891.000000,0.000000


In [59]:
historico_comerci[['city_id', 'installments', 'merchant_category_id', 'month_lag']].describe()
# month_lag em historico_cartoes é abaixo de zero e em historico_comerci é mínimo 1
# month_lag = atraso do mês até a data de referência
# purchase_amount o valor máximo é superior em historico_cartoes
# purchase_amount é valor em normalidade

,city_id,installments,merchant_category_id,month_lag
count,1.776146e+06,1.776146e+06,1.776146e+06,1.776146e+06
mean,1.413583e+02,6.769139e-01,4.258958e+02,1.473437e+00
std,9.936808e+01,1.506255e+00,2.442267e+02,4.992941e-01
min,1.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00
25%,6.900000e+01,0.000000e+00,2.780000e+02,1.000000e+00
50%,1.250000e+02,1.000000e+00,3.670000e+02,1.000000e+00
75%,2.130000e+02,1.000000e+00,6.830000e+02,2.000000e+00
max,3.470000e+02,9.990000e+02,8.910000e+02,2.000000e+00


In [60]:
historico_cartoes.month_lag.value_counts()

-2     20203
-1     18806
-3     17357
 0     16729
-4     16682
-5     15970
-6     14692
-7     13289
-8     10378
-9      9897
-10     8550
-11     8466
-12     6700
-13     6086
Name: month_lag, dtype: int64

In [61]:
historico_comerci.month_lag.value_counts()

1    935252
2    840894
Name: month_lag, dtype: int64

In [62]:
# Colunas purchase_amount, state_id, subsector_id
historico_cartoes[['purchase_amount', 'state_id', 'subsector_id']].describe()

,purchase_amount,state_id,subsector_id
count,183805.000000,183805.000000,183805.000000
mean,-0.466595,11.720563,27.985659
std,34.167408,5.397268,9.311017
min,-0.746893,1.000000,1.000000
25%,-0.722084,9.000000,19.000000
50%,-0.694315,9.000000,33.000000
75%,-0.625989,16.000000,34.000000
max,11269.666518,24.000000,41.000000


In [63]:
historico_comerci[['purchase_amount', 'state_id', 'subsector_id']].describe()

,purchase_amount,state_id,subsector_id
count,1.776146e+06,1.776146e+06,1.776146e+06
mean,-5.780843e-01,1.159748e+01,2.626767e+01
std,5.904739e-01,5.443451e+00,1.004141e+01
min,-7.468928e-01,1.000000e+00,1.000000e+00
25%,-7.168548e-01,9.000000e+00,1.900000e+01
50%,-6.792885e-01,9.000000e+00,2.900000e+01
75%,-5.966427e-01,1.600000e+01,3.400000e+01
max,2.631575e+02,2.400000e+01,4.100000e+01


In [64]:
# month_lag possui o mesmo conceito mas o ponto de visto é diferente, enquanto em historico_cartoes prevalece valores
# negativos, em historico_comerci só existem valores positivos. Comparar o valor de purchase_amount com referência em
# month_lag
historico_cartoes[historico_cartoes['month_lag'] < 6]['purchase_amount'].describe()

count    183805.000000
mean         -0.466595
std          34.167408
min          -0.746893
25%          -0.722084
50%          -0.694315
75%          -0.625989
max       11269.666518
Name: purchase_amount, dtype: float64

In [65]:
# Quantidade de observações com purchase_amount maior que zero:
print(historico_cartoes[historico_cartoes['purchase_amount'] >= 0]['card_id'].count())
print(historico_comerci[historico_comerci['purchase_amount'] >= 0]['card_id'].count())

4260
59565


In [66]:
# Quantidade de observações com purchase_amount menor que zero:
print(historico_cartoes[historico_cartoes['purchase_amount'] < 0]['card_id'].count())
print(historico_comerci[historico_comerci['purchase_amount'] < 0]['card_id'].count())

179545
1716581


In [67]:
# Compatibilizar os valores dos dataframes
historico_comerci['month_lag'] = historico_comerci['month_lag'].map({1: -1, 2: -2})

In [68]:
# Concatenar historico_cartoes e historico_comerci
historico_cartoes_comerci = pd.concat([historico_cartoes, historico_comerci])

In [69]:
# Criar nova coluna para month_lag com valores positivos, visando possível normalização
historico_cartoes_comerci['delay_of_the_month'] = historico_cartoes_comerci['month_lag'] * (-1)
historico_cartoes_comerci.drop(['month_lag'], axis = 1, inplace = True)

In [70]:
historico_cartoes_comerci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959951 entries, 0 to 1963030
Data columns (total 19 columns):
card_id                       object
city_id                       int64
installments                  int64
merchant_category_id          int64
merchant_id                   object
purchase_amount               float64
state_id                      int64
subsector_id                  int64
category_2_1.0                uint8
category_2_2.0                uint8
category_2_3.0                uint8
category_2_4.0                uint8
category_2_5.0                uint8
category_3_A                  uint8
category_3_B                  uint8
category_3_C                  uint8
Unauthorized_card_purchase    int64
Purchase_transaction_date     datetime64[ns]
delay_of_the_month            int64
dtypes: datetime64[ns](1), float64(1), int64(7), object(2), uint8(8)
memory usage: 194.4+ MB


<h3>Preparar merge historico_cartoes_comerci com comerciantes_file</h3>

In [71]:
comerciantes_file.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,state_id,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,1,9,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,1,16,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,1,5,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
16,M_ID_c929bb59af,9514,2,20,-0.057471,-0.057471,69667.00,1.000000,3,69667.00,1.000000,3,69667.00,1.000000,3,1,20,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
17,M_ID_dd3ae3de10,30534,278,37,-0.057471,-0.057471,5180.00,4.000000,3,9830.00,4.166667,6,15826.00,5.600000,10,0,23,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1


In [72]:
historico_cartoes_comerci.head()

,card_id,city_id,installments,merchant_category_id,merchant_id,purchase_amount,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,Unauthorized_card_purchase,Purchase_transaction_date,delay_of_the_month
0,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-0.733128,16,16,1,0,0,0,0,1,0,0,0,2017-07-15 12:10:45,7
1,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-0.720386,16,37,1,0,0,0,0,1,0,0,0,2017-08-09 22:04:29,6
2,C_ID_4e6213e9bc,88,0,560,M_ID_e6d5ae8ea6,-0.735352,16,34,1,0,0,0,0,1,0,0,0,2017-09-02 10:06:26,5
3,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-0.722865,16,37,1,0,0,0,0,1,0,0,0,2017-03-10 01:14:19,11
4,C_ID_4e6213e9bc,333,0,80,M_ID_50af771f8d,-0.734887,9,37,1,0,0,0,0,1,0,0,0,2018-02-24 08:45:05,0


In [73]:
# Merge será feito pela variável merchant_id. Remover as variáveis id exceto merchant_id:
comerciantes_file.drop(['merchant_group_id', 'merchant_category_id', 'subsector_id', 'state_id'], axis = 1, inplace = True)

In [74]:
# Em histórico geral, remover as variáveis id, exceto merchant_id e card_id
historico_cartoes_comerci.drop(['city_id', 'merchant_category_id', 'state_id', 'subsector_id'], axis = 1, inplace = True)

In [75]:
comerciantes_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322802 entries, 0 to 334695
Data columns (total 29 columns):
merchant_id                      322802 non-null object
numerical_1                      322802 non-null float64
numerical_2                      322802 non-null float64
avg_sales_lag3                   322802 non-null float64
avg_purchases_lag3               322802 non-null float64
active_months_lag3               322802 non-null int64
avg_sales_lag6                   322802 non-null float64
avg_purchases_lag6               322802 non-null float64
active_months_lag6               322802 non-null int64
avg_sales_lag12                  322802 non-null float64
avg_purchases_lag12              322802 non-null float64
active_months_lag12              322802 non-null int64
category_4                       322802 non-null int64
category_1_N                     322802 non-null uint8
category_2_1.0                   322802 non-null uint8
category_2_2.0                   322802 non-nul

In [76]:
historico_cartoes_comerci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1959951 entries, 0 to 1963030
Data columns (total 15 columns):
card_id                       object
installments                  int64
merchant_id                   object
purchase_amount               float64
category_2_1.0                uint8
category_2_2.0                uint8
category_2_3.0                uint8
category_2_4.0                uint8
category_2_5.0                uint8
category_3_A                  uint8
category_3_B                  uint8
category_3_C                  uint8
Unauthorized_card_purchase    int64
Purchase_transaction_date     datetime64[ns]
delay_of_the_month            int64
dtypes: datetime64[ns](1), float64(1), int64(3), object(2), uint8(8)
memory usage: 134.6+ MB


In [77]:
# comerciantes_file - quantidade de id unico
comerciantes_file.merchant_id.nunique()

322763

In [78]:
# comerciantes_file - quantidade de id's que aparecem mais de uma vez:
comerciantes_file.duplicated(subset=['merchant_id']).sum()

39

In [79]:
# merchant_id com mais de uma ocorrência:
comerciantes_file['merchant_id'].value_counts().loc[lambda x : x>1]

M_ID_992a180b15    4
M_ID_30340088f2    4
M_ID_1802942aaf    4
M_ID_bd49e37dda    4
M_ID_ef233cff26    4
M_ID_42697d5d44    4
M_ID_dbbf07ebf0    4
M_ID_6017075769    2
M_ID_32d73b8ecf    2
M_ID_fea38c640b    2
M_ID_7483dde87d    2
M_ID_26d4fadb60    2
M_ID_508ecbdcde    2
M_ID_6c144405a0    2
M_ID_a8767b29ef    2
M_ID_d2b5d4418d    2
M_ID_645a6af169    2
M_ID_0c4018d3a0    2
M_ID_07a5671b4c    2
M_ID_49daf8a838    2
M_ID_c0b712e11a    2
M_ID_9b0ef314cf    2
M_ID_b794b9d9e8    2
M_ID_28c2aace87    2
M_ID_ae9fe1605a    2
Name: merchant_id, dtype: int64

Estratégia para remover os duplicados (fazer caso a caso):
<ul>
  <li>remover o menor valor em active_months_lag do maior para o menor período</li>
  <li>remover o menor valor em avg_purchases_lag do maior para o menor período</li>
  <li>remover o menor valor em avg_sales_lag do maior para o menor período</li>
</ul>

In [80]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_992a180b15']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
221180,M_ID_992a180b15,-0.057471,-0.057471,5.56,2.522242,2,5.56,2.522242,2,5.56,2.522242,2,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
221181,M_ID_992a180b15,-0.057471,-0.057471,5.56,2.522242,2,5.56,2.522242,2,5.56,2.522242,2,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
221182,M_ID_992a180b15,-0.057471,-0.057471,1.42,1.104119,3,1.37,1.095421,6,1.66,1.229726,12,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0
221183,M_ID_992a180b15,-0.057471,-0.057471,1.42,1.104119,3,1.37,1.095421,6,1.66,1.229726,12,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0


In [81]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_992a180b15') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [82]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_bd49e37dda']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
3392,M_ID_bd49e37dda,-0.057471,-0.057471,60.09,9.333333,3,104.82,13.800000,5,104.82,13.800000,5,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3393,M_ID_bd49e37dda,-0.057471,-0.057471,60.09,9.333333,3,104.82,13.800000,5,104.82,13.800000,5,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3394,M_ID_bd49e37dda,-0.057471,-0.057471,1.35,1.526667,3,1.23,1.546667,6,1.23,1.546667,6,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3395,M_ID_bd49e37dda,-0.057471,-0.057471,1.35,1.526667,3,1.23,1.546667,6,1.23,1.546667,6,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [83]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_bd49e37dda') & (comerciantes_file['active_months_lag12'] < 6)].index, axis = 0, inplace = True)

In [84]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_dbbf07ebf0']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
7584,M_ID_dbbf07ebf0,-0.057471,-0.057471,52.08,45.333333,3,75.36,68.000000,6,82.95,70.4500,10,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
7585,M_ID_dbbf07ebf0,-0.057471,-0.057471,52.08,45.333333,3,75.36,68.000000,6,82.95,70.4500,10,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
7586,M_ID_dbbf07ebf0,-0.047556,-0.047556,1.12,1.132979,3,1.23,1.234043,6,1.30,1.3125,12,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
7587,M_ID_dbbf07ebf0,-0.047556,-0.047556,1.12,1.132979,3,1.23,1.234043,6,1.30,1.3125,12,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [85]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_dbbf07ebf0') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [86]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_1802942aaf']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
334070,M_ID_1802942aaf,-0.047556,-0.047556,0.94,0.954538,3,0.85,0.868956,6,0.57,0.603051,12,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
334071,M_ID_1802942aaf,-0.047556,-0.047556,0.94,0.954538,3,0.85,0.868956,6,0.57,0.603051,12,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
334072,M_ID_1802942aaf,-0.027726,-0.027726,1.01,1.064999,3,0.98,1.102663,6,0.97,1.269495,12,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
334073,M_ID_1802942aaf,-0.027726,-0.027726,1.01,1.064999,3,0.98,1.102663,6,0.97,1.269495,12,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [87]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_1802942aaf') & (comerciantes_file['avg_purchases_lag12'] < 1)].index, axis = 0, inplace = True)

In [88]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_42697d5d44']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
330957,M_ID_42697d5d44,-0.057471,-0.057471,0.88,0.906226,3,0.74,0.827259,6,0.66,0.813497,12,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
330958,M_ID_42697d5d44,-0.057471,-0.057471,0.88,0.906226,3,0.74,0.827259,6,0.66,0.813497,12,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
330959,M_ID_42697d5d44,-0.027726,-0.057471,0.88,0.903420,3,0.63,0.704480,6,0.47,0.561476,12,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
330960,M_ID_42697d5d44,-0.027726,-0.057471,0.88,0.903420,3,0.63,0.704480,6,0.47,0.561476,12,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [89]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_42697d5d44') & (comerciantes_file['avg_purchases_lag12'] < 0.9)].index, axis = 0, inplace = True)

In [90]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_ef233cff26']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
4181,M_ID_ef233cff26,-0.047556,-0.057471,322.44,76.555556,3,817.53,162.055556,6,778.20,149.592593,9,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4182,M_ID_ef233cff26,-0.047556,-0.057471,322.44,76.555556,3,817.53,162.055556,6,778.20,149.592593,9,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4183,M_ID_ef233cff26,-0.057471,-0.057471,4.74,4.077922,3,4.98,4.174242,6,4.25,3.509740,8,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4184,M_ID_ef233cff26,-0.057471,-0.057471,4.74,4.077922,3,4.98,4.174242,6,4.25,3.509740,8,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [91]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_ef233cff26') & (comerciantes_file['avg_purchases_lag12'] < 0.9)].index, axis = 0, inplace = True)

In [92]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_30340088f2']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
29464,M_ID_30340088f2,-0.057471,-0.057471,1.00,1.000000,1,1.00,1.000000,1,1.0,1.000000,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
29465,M_ID_30340088f2,-0.057471,-0.057471,1.00,1.000000,1,1.00,1.000000,1,1.0,1.000000,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
29466,M_ID_30340088f2,0.111083,-0.057471,0.84,1.087449,3,0.84,1.089712,6,0.9,1.107765,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
29467,M_ID_30340088f2,0.111083,-0.057471,0.84,1.087449,3,0.84,1.089712,6,0.9,1.107765,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [93]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_30340088f2') & (comerciantes_file['avg_purchases_lag12'] < 0.9)].index, axis = 0, inplace = True)

In [94]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_30340088f2']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
29464,M_ID_30340088f2,-0.057471,-0.057471,1.00,1.000000,1,1.00,1.000000,1,1.0,1.000000,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
29465,M_ID_30340088f2,-0.057471,-0.057471,1.00,1.000000,1,1.00,1.000000,1,1.0,1.000000,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
29466,M_ID_30340088f2,0.111083,-0.057471,0.84,1.087449,3,0.84,1.089712,6,0.9,1.107765,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
29467,M_ID_30340088f2,0.111083,-0.057471,0.84,1.087449,3,0.84,1.089712,6,0.9,1.107765,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [95]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_30340088f2') & (comerciantes_file['avg_purchases_lag12'] < 1.1)].index, axis = 0, inplace = True)

In [96]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_ef233cff26']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
4181,M_ID_ef233cff26,-0.047556,-0.057471,322.44,76.555556,3,817.53,162.055556,6,778.20,149.592593,9,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4182,M_ID_ef233cff26,-0.047556,-0.057471,322.44,76.555556,3,817.53,162.055556,6,778.20,149.592593,9,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4183,M_ID_ef233cff26,-0.057471,-0.057471,4.74,4.077922,3,4.98,4.174242,6,4.25,3.509740,8,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4184,M_ID_ef233cff26,-0.057471,-0.057471,4.74,4.077922,3,4.98,4.174242,6,4.25,3.509740,8,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [97]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_ef233cff26') & (comerciantes_file['avg_purchases_lag12'] < 100)].index, axis = 0, inplace = True)

In [98]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_a8767b29ef']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
52876,M_ID_a8767b29ef,-0.057471,-0.057471,4.39,11.000000,2,4.39,11.000000,2,4.39,11.000000,2,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
52877,M_ID_a8767b29ef,0.279636,0.279636,1.84,1.629037,3,1.58,1.406444,6,1.59,1.387683,12,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0


In [99]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_a8767b29ef') & (comerciantes_file['avg_purchases_lag12'] < 10)].index, axis = 0, inplace = True)

In [100]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_6017075769']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
42514,M_ID_6017075769,-0.057471,-0.057471,0.78,1.123457,3,1.29,1.746914,6,1.14,1.546296,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
42515,M_ID_6017075769,0.537424,0.537424,1.00,1.075047,3,1.03,1.101063,6,0.99,1.034001,12,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [101]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_6017075769') & (comerciantes_file['avg_purchases_lag12'] < 1.5)].index, axis = 0, inplace = True)

In [102]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_c0b712e11a']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
53,M_ID_c0b712e11a,-0.057471,-0.057471,1.00,1.000000,1,1.00,1.000000,1,1.0,1.000000,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
54,M_ID_c0b712e11a,-0.007896,-0.007896,1.01,0.950617,3,1.01,1.097119,6,1.2,1.190741,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [103]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_c0b712e11a') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [104]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_d2b5d4418d']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
326908,M_ID_d2b5d4418d,-0.057471,-0.057471,0.96,0.956377,3,0.96,1.020057,6,1.26,1.298293,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
326909,M_ID_d2b5d4418d,-0.047556,-0.047556,0.92,0.971041,3,0.83,0.923229,6,0.87,0.937825,12,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0


In [105]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_d2b5d4418d') & (comerciantes_file['avg_purchases_lag12'] < 1)].index, axis = 0, inplace = True)

In [106]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_7483dde87d']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
152187,M_ID_7483dde87d,-0.027726,-0.027726,0.98,1.027778,3,0.98,1.027778,3,0.98,1.027778,3,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
152188,M_ID_7483dde87d,1.181892,1.171977,1.11,1.099736,3,1.19,1.135586,6,1.22,1.152302,12,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0


In [107]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_7483dde87d') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [108]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_07a5671b4c']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
70493,M_ID_07a5671b4c,-0.057471,-0.057471,1.00,1.000000,1,1.0,1.000000,1,1.00,1.000000,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
70494,M_ID_07a5671b4c,0.031763,0.031763,1.26,1.142222,3,1.2,1.253333,6,1.15,1.262831,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [109]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_07a5671b4c') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [110]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_9b0ef314cf']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
132061,M_ID_9b0ef314cf,-0.057471,-0.057471,1.84,1.666667,3,1.54,1.353333,4,1.54,1.353333,4,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
132062,M_ID_9b0ef314cf,0.041678,0.041678,1.37,1.156790,3,1.37,1.133951,6,1.47,1.090123,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [111]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_9b0ef314cf') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [112]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_6c144405a0']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
197477,M_ID_6c144405a0,-0.027726,-0.027726,0.70,0.716312,3,0.91,1.066667,6,0.91,1.182506,9,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
197478,M_ID_6c144405a0,1.162063,1.152148,1.19,1.184584,3,1.34,1.316539,6,1.33,1.327270,12,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0


In [113]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_6c144405a0') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [114]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_645a6af169']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
47803,M_ID_645a6af169,-0.057471,-0.057471,1.69,0.791667,3,2.62,1.812500,6,2.30,1.458333,12,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
47804,M_ID_645a6af169,-0.057471,-0.057471,0.89,1.111111,3,1.54,1.555556,6,1.55,2.000000,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [115]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_645a6af169') & (comerciantes_file['avg_purchases_lag12'] < 2)].index, axis = 0, inplace = True)

In [116]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_49daf8a838']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
85377,M_ID_49daf8a838,-0.047556,-0.047556,1.18,0.735931,3,1.18,0.735931,3,1.18,0.735931,3,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
85378,M_ID_49daf8a838,0.101168,0.101168,1.27,1.019540,3,1.46,1.111494,6,1.47,1.093021,12,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [117]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_49daf8a838') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [118]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_32d73b8ecf']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
50528,M_ID_32d73b8ecf,-0.057471,-0.057471,1.10,1.285714,3,1.16,1.642857,6,2.17,2.973214,12,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
50529,M_ID_32d73b8ecf,11.523138,11.265350,1.15,1.142330,3,1.27,1.238683,6,1.41,1.336988,12,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [119]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_32d73b8ecf') & (comerciantes_file['avg_purchases_lag12'] < 2)].index, axis = 0, inplace = True)

In [120]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_b794b9d9e8']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
71809,M_ID_b794b9d9e8,-0.047556,-0.047556,1.59,3.098039,3,2.02,4.225490,6,2.26,4.198529,12,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
71810,M_ID_b794b9d9e8,136.827699,119.278318,1.15,1.174350,3,1.17,1.200552,6,1.21,1.219879,12,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [121]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_b794b9d9e8') & (comerciantes_file['avg_purchases_lag12'] < 4)].index, axis = 0, inplace = True)

In [122]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_fea38c640b']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
83151,M_ID_fea38c640b,-0.057471,-0.057471,0.47,0.666667,3,0.47,0.666667,3,0.47,0.666667,3,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
83152,M_ID_fea38c640b,0.616743,0.616743,0.84,0.841643,3,0.84,0.859307,6,0.86,0.889253,12,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0


In [123]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_fea38c640b') & (comerciantes_file['active_months_lag12'] < 4)].index, axis = 0, inplace = True)

In [124]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_28c2aace87']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
84269,M_ID_28c2aace87,-0.057471,-0.057471,0.66,1.237037,3,3.83,2.868519,6,6.16,4.132323,11,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
84270,M_ID_28c2aace87,0.914190,0.914190,1.08,1.454729,3,1.22,1.595643,6,1.22,1.558444,12,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [125]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_28c2aace87') & (comerciantes_file['avg_purchases_lag12'] < 4)].index, axis = 0, inplace = True)

In [126]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_ae9fe1605a']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
110010,M_ID_ae9fe1605a,-0.057471,-0.057471,0.71,0.819672,3,0.57,0.546448,6,0.49,0.470726,7,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
110011,M_ID_ae9fe1605a,128.628073,108.897413,1.06,1.070994,3,1.07,1.077628,6,1.06,1.055074,12,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [127]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_ae9fe1605a') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [128]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_0c4018d3a0']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
8172,M_ID_0c4018d3a0,-0.057471,-0.057471,1.40,1.000000,3,4.47,2.444444,6,7.44,3.416667,12,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
8173,M_ID_0c4018d3a0,-0.057471,-0.057471,0.51,0.532864,2,0.51,0.532864,2,0.51,0.532864,2,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0


In [129]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_0c4018d3a0') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [130]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_26d4fadb60']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
66836,M_ID_26d4fadb60,-0.057471,-0.057471,0.86,0.833333,3,0.81,0.739583,6,0.68,0.611111,9,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
66837,M_ID_26d4fadb60,24.243960,23.847364,1.11,1.128992,3,1.14,1.124452,6,1.18,1.162055,12,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0


In [131]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_26d4fadb60') & (comerciantes_file['active_months_lag12'] < 12)].index, axis = 0, inplace = True)

In [132]:
comerciantes_file[comerciantes_file['merchant_id'] == 'M_ID_508ecbdcde']

,merchant_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
11363,M_ID_508ecbdcde,-0.057471,-0.057471,45.61,173.666667,3,71.74,261.166667,6,74.14,299.083333,12,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
11364,M_ID_508ecbdcde,25.671706,24.313364,1.19,1.205294,3,1.26,1.286341,6,1.30,1.309678,12,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [133]:
comerciantes_file.drop(comerciantes_file.loc[(comerciantes_file['merchant_id'] == 'M_ID_508ecbdcde') & (comerciantes_file['avg_purchases_lag12'] < 12)].index, axis = 0, inplace = True)

In [134]:
# merchant_id com mais de uma ocorrência:
comerciantes_file['merchant_id'].value_counts().loc[lambda x : x>1]

M_ID_bd49e37dda    2
M_ID_ef233cff26    2
M_ID_dbbf07ebf0    2
M_ID_992a180b15    2
M_ID_30340088f2    2
M_ID_1802942aaf    2
Name: merchant_id, dtype: int64

In [135]:
# Após todo este esforço braçal ainda sobraram 6 registros duplicados. Estes podem ser removidos com um único comando
comerciantes_file = comerciantes_file.drop_duplicates(subset='merchant_id', keep='first')

In [136]:
# Resolvido o problema da duplicidade, o merge pode ser efetivado
# Alterar o índice para merchant_id
comerciantes_file.set_index('merchant_id', inplace=True)
historico_cartoes_comerci.set_index('merchant_id',  inplace=True)

In [137]:
comerciantes_file.head(10)

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
M_ID_838061e48c,-0.057471,-0.057471,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
M_ID_9339d880ad,-0.057471,-0.057471,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
M_ID_e726bbae1e,-0.057471,-0.057471,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
M_ID_c929bb59af,-0.057471,-0.057471,69667.00,1.000000,3,69667.00,1.000000,3,69667.00,1.000000,3,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
M_ID_dd3ae3de10,-0.057471,-0.057471,5180.00,4.000000,3,9830.00,4.166667,6,15826.00,5.600000,10,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
M_ID_dc671140d0,-0.057471,-0.057471,7133.00,4.666667,3,17566.00,4.500000,6,19594.00,6.000000,9,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
M_ID_7551d55443,-0.057471,-0.057471,109733.00,18.666667,3,118433.00,19.000000,6,217857.00,35.416667,12,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
M_ID_a8effe9197,-0.057471,-0.057471,3897.00,1.333333,3,2924.00,1.250000,4,2924.00,1.250000,4,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
M_ID_39643c7d93,-0.057471,-0.057471,608433.00,5.666667,3,776683.00,7.000000,6,749804.00,6.000000,12,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [138]:
historico_cartoes_comerci.head(10)

,card_id,installments,purchase_amount,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,Unauthorized_card_purchase,Purchase_transaction_date,delay_of_the_month
merchant_id,,,,,,,,,,,,,,
M_ID_86ec983688,C_ID_4e6213e9bc,0,-0.733128,1,0,0,0,0,1,0,0,0,2017-07-15 12:10:45,7
M_ID_979ed661fc,C_ID_4e6213e9bc,0,-0.720386,1,0,0,0,0,1,0,0,0,2017-08-09 22:04:29,6
M_ID_e6d5ae8ea6,C_ID_4e6213e9bc,0,-0.735352,1,0,0,0,0,1,0,0,0,2017-09-02 10:06:26,5
M_ID_e020e9b302,C_ID_4e6213e9bc,0,-0.722865,1,0,0,0,0,1,0,0,0,2017-03-10 01:14:19,11
M_ID_50af771f8d,C_ID_4e6213e9bc,0,-0.734887,1,0,0,0,0,1,0,0,0,2018-02-24 08:45:05,0
M_ID_5e8220e564,C_ID_4e6213e9bc,0,-0.716855,1,0,0,0,0,1,0,0,0,2017-03-21 00:10:51,11
M_ID_9d41786a50,C_ID_4e6213e9bc,0,-0.657049,1,0,0,0,0,1,0,0,0,2017-11-18 20:05:55,3
M_ID_979ed661fc,C_ID_4e6213e9bc,0,-0.737967,1,0,0,0,0,1,0,0,0,2017-06-01 22:02:56,8
M_ID_74ba14b5fc,C_ID_4e6213e9bc,0,-0.715352,1,0,0,0,0,1,0,0,0,2017-03-16 15:41:22,11


In [139]:
# Criar buckets para a variável delay_of_the_month
bins = [0, 1, 5, 10, 13]
historico_cartoes_comerci['months_lag'] = np.searchsorted(bins, historico_cartoes_comerci['delay_of_the_month'].values)

In [140]:
# Remover a variável delay_of_the_month
historico_cartoes_comerci.drop('delay_of_the_month', axis = 'columns', inplace = True)

In [141]:
# Criar dummies para a nova coluna months_lag:
historico_cartoes_comerci = pd.get_dummies(historico_cartoes_comerci, columns = ['months_lag'])

In [142]:
# Executar o merge:
df_historico_e_caract = pd.merge(comerciantes_file, historico_cartoes_comerci, how='left', left_index=True, right_index=True)

In [143]:
if (df_historico_e_caract.purchase_amount.isna().sum()) > 0:
    df_historico_e_caract.dropna(inplace=True)
#
# Verificar presença de valores NAN
df_historico_e_caract.isna().sum()

numerical_1                      0
numerical_2                      0
avg_sales_lag3                   0
avg_purchases_lag3               0
active_months_lag3               0
avg_sales_lag6                   0
avg_purchases_lag6               0
active_months_lag6               0
avg_sales_lag12                  0
avg_purchases_lag12              0
active_months_lag12              0
category_4                       0
category_1_N                     0
category_2_1.0_x                 0
category_2_2.0_x                 0
category_2_3.0_x                 0
category_2_4.0_x                 0
category_2_5.0_x                 0
most_recent_sales_range_A        0
most_recent_sales_range_B        0
most_recent_sales_range_C        0
most_recent_sales_range_D        0
most_recent_sales_range_E        0
most_recent_purchases_range_A    0
most_recent_purchases_range_B    0
most_recent_purchases_range_C    0
most_recent_purchases_range_D    0
most_recent_purchases_range_E    0
card_id             

<h3>Preparar merge dos dataframes de treino e teste com df_historico_e_caract</h3>

In [144]:
# Excluir índice e variável
df_historico_e_caract = df_historico_e_caract.reset_index(drop = True)

In [145]:
df_historico_e_caract.head()

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_1_N,category_2_1.0_x,category_2_2.0_x,category_2_3.0_x,category_2_4.0_x,category_2_5.0_x,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,card_id,installments,purchase_amount,category_2_1.0_y,category_2_2.0_y,category_2_3.0_y,category_2_4.0_y,category_2_5.0_y,category_3_A,category_3_B,category_3_C,Unauthorized_card_purchase,Purchase_transaction_date,months_lag_0,months_lag_1,months_lag_2,months_lag_3,months_lag_4
0,-0.057471,-0.057471,1.29,1.880342,3,1.26,2.228632,6,1.08,2.089744,12,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,C_ID_206afe601c,1.0,-0.662759,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2018-03-17 16:07:34,0.0,1.0,0.0,0.0,0.0
1,-0.057471,-0.057471,0.99,0.997832,3,1.04,1.002168,6,1.24,1.154878,12,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,C_ID_a1f21dc8ca,0.0,-0.704834,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-12-21 10:43:34,0.0,0.0,1.0,0.0,0.0
2,-0.057471,-0.057471,0.99,0.997832,3,1.04,1.002168,6,1.24,1.154878,12,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,C_ID_a1f21dc8ca,0.0,-0.676283,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-07 17:38:17,0.0,0.0,1.0,0.0,0.0
3,-0.057471,-0.057471,0.99,0.997832,3,1.04,1.002168,6,1.24,1.154878,12,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,C_ID_c4e202ded8,0.0,-0.687553,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-03-02 18:03:41,0.0,1.0,0.0,0.0,0.0
4,-0.057471,-0.057471,0.99,0.997832,3,1.04,1.002168,6,1.24,1.154878,12,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,C_ID_ce4ab774c4,0.0,-0.697170,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018-03-15 15:42:50,0.0,1.0,0.0,0.0,0.0


In [146]:
df_historico_e_caract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1833199 entries, 0 to 1833198
Data columns (total 46 columns):
numerical_1                      float64
numerical_2                      float64
avg_sales_lag3                   float64
avg_purchases_lag3               float64
active_months_lag3               int64
avg_sales_lag6                   float64
avg_purchases_lag6               float64
active_months_lag6               int64
avg_sales_lag12                  float64
avg_purchases_lag12              float64
active_months_lag12              int64
category_4                       int64
category_1_N                     uint8
category_2_1.0_x                 uint8
category_2_2.0_x                 uint8
category_2_3.0_x                 uint8
category_2_4.0_x                 uint8
category_2_5.0_x                 uint8
most_recent_sales_range_A        uint8
most_recent_sales_range_B        uint8
most_recent_sales_range_C        uint8
most_recent_sales_range_D        uint8
most_recent_s

In [147]:
# Criar a função que vai gerar as colunas de data min e data max
def create_date_columns(x):
    resultado = pd.DataFrame()
    resultado['denied_purchase?'] = pd.Series(['Y' if 0 in x['category_2_4.0_x'].unique() else 'N'])
    resultado['oldest_Date']      = x['Purchase_transaction_date'].min()
    resultado['latest_Date']      = x['Purchase_transaction_date'].max()
    return resultado

In [148]:
# Aplicar a função create_date_columns sobre df_historico_e_caract
# e gerar dataframe df_card_dates com índice em card_id
df_card_dates = df_historico_e_caract.groupby('card_id').apply(create_date_columns)
df_card_dates = df_card_dates.reset_index().drop('level_1', axis=1).set_index('card_id')

In [149]:
df_card_dates.drop('denied_purchase?', axis = 'columns', inplace = True)

In [150]:
# Alterar o índice para card_id
train_file.set_index('card_id', inplace = True)
test_file.set_index('card_id',  inplace = True)
df_historico_e_caract.set_index('card_id', inplace = True)

In [151]:
df_card_dates.head()

,oldest_Date,latest_Date
card_id,,
C_ID_00007093c1,2018-04-03 11:13:35,2018-04-03 11:13:35
C_ID_0001238066,2018-03-01 16:48:27,2018-04-29 01:19:02
C_ID_0001506ef0,2018-03-22 09:14:30,2018-03-22 09:14:30
C_ID_0001793786,2017-11-15 15:44:20,2017-12-31 17:35:56
C_ID_000183fdda,2018-03-02 12:26:26,2018-04-26 13:46:18


In [152]:
train_file.head()

,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,date_Card_Activation,alvo
card_id,,,,,,,,,,,
C_ID_92a2005557,0,0,0,0,1,0,1,0,1,2017-06-01,-0.820283
C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,2017-01-01,0.392913
C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,2016-08-01,0.688056
C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,2017-09-01,0.142495
C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,2017-11-01,-0.159749


In [153]:
# remover os índices
df_card_dates.reset_index(inplace=True)
df_historico_e_caract.reset_index(inplace = True)

In [154]:
# Executar os primeiros merges:
df_treino = pd.merge(train_file, df_card_dates, how='left', on='card_id')
df_teste  = pd.merge(test_file,  df_card_dates, how='left', on='card_id')

In [155]:
df_treino.card_id.count()

201917

In [156]:
df_teste.card_id.count()

123623

In [157]:
df_treino.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,date_Card_Activation,alvo,oldest_Date,latest_Date
0,C_ID_92a2005557,0,0,0,0,1,0,1,0,1,2017-06-01,-0.820283,2018-03-05 14:04:36,2018-04-29 11:23:05
1,C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,2017-01-01,0.392913,2018-02-01 17:07:54,2018-03-30 06:48:26
2,C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,2016-08-01,0.688056,2018-04-28 17:43:11,2018-04-28 17:43:11
3,C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,2017-09-01,0.142495,2018-03-07 11:55:06,2018-04-17 07:15:40
4,C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,2017-11-01,-0.159749,2018-03-02 11:55:43,2018-04-28 18:50:25


In [158]:
df_teste.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,date_Card_Activation,oldest_Date,latest_Date
0,C_ID_0ab67a22ab,0,0,1,0,0,0,0,1,1,2017-04-01,2018-02-03 09:44:29,2018-02-28 19:08:52
1,C_ID_130fd0cbdd,0,1,0,0,0,0,0,1,0,2017-01-01,2018-03-03 12:18:48,2018-04-20 18:00:17
2,C_ID_b709037bc5,0,0,0,0,1,1,0,0,1,2017-08-01,2018-03-01 14:51:33,2018-03-01 14:51:33
3,C_ID_d27d835a9f,0,1,0,0,0,1,0,0,0,2017-12-01,2018-03-04 11:06:29,2018-04-17 21:22:23
4,C_ID_2b5e3df5c2,0,0,0,0,1,1,0,0,1,2015-12-01,2018-03-04 03:01:37,2018-03-09 12:03:47


In [159]:
# Criar colunas com resultado de cálculos com data de ativação (em anos)
df_treino['oldest_Activation'] = ((df_treino['oldest_Date'] - df_treino['date_Card_Activation']).dt.days) / 365
df_treino['latest_Activation'] = ((df_treino['latest_Date'] - df_treino['date_Card_Activation']).dt.days) / 365
df_teste['oldest_Activation']  = ((df_teste['oldest_Date'] - df_teste['date_Card_Activation']).dt.days) / 365
df_teste['latest_Activation']  = ((df_teste['latest_Date'] - df_teste['date_Card_Activation']).dt.days) / 365

In [160]:
# Excluir as colunas de data:
df_treino.drop(['date_Card_Activation', 'oldest_Date', 'latest_Date'], axis=1, inplace = True)
df_teste.drop(['date_Card_Activation', 'oldest_Date', 'latest_Date'], axis=1, inplace = True)

In [161]:
df_treino.card_id.count()

201917

In [162]:
df_teste.card_id.count()

123623

In [163]:
# Verificar a presença de valores NA:
df_treino.isna().sum()

card_id                  0
feature_1_1              0
feature_1_2              0
feature_1_3              0
feature_1_4              0
feature_1_5              0
feature_2_1              0
feature_2_2              0
feature_2_3              0
feature_3_1_0            0
alvo                     0
oldest_Activation    34392
latest_Activation    34392
dtype: int64

In [164]:
df_teste.isna().sum()

card_id                  0
feature_1_1              0
feature_1_2              0
feature_1_3              0
feature_1_4              0
feature_1_5              0
feature_2_1              0
feature_2_2              0
feature_2_3              0
feature_3_1_0            0
oldest_Activation    21396
latest_Activation    21396
dtype: int64

In [185]:
df_treino.head()

,card_id,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_1_0,alvo,oldest_Activation,latest_Activation
0,C_ID_92a2005557,0,0,0,0,1,0,1,0,1,-0.820283,0.758904,0.909589
1,C_ID_3d0044924f,0,0,0,1,0,1,0,0,0,0.392913,1.084932,1.241096
2,C_ID_d639edf6cd,0,1,0,0,0,0,1,0,0,0.688056,1.739726,1.739726
3,C_ID_186d6a6901,0,0,0,1,0,0,0,1,0,0.142495,0.512329,0.624658
4,C_ID_cdbd2c0db2,1,0,0,0,0,0,0,1,0,-0.159749,0.331507,0.487671


In [169]:
# Substituir os valores NA por zero
df_treino_v2 = df_treino.fillna(0, axis = 0)
df_teste_v2  = df_teste.fillna(0, axis  = 0)

In [171]:
df_treino_v2.card_id.count()

201917

In [172]:
df_teste_v2.card_id.count()

123623

In [187]:
df_treino_v2.isna().sum()

card_id              0
feature_1_1          0
feature_1_2          0
feature_1_3          0
feature_1_4          0
feature_1_5          0
feature_2_1          0
feature_2_2          0
feature_2_3          0
feature_3_1_0        0
alvo                 0
oldest_Activation    0
latest_Activation    0
dtype: int64

In [188]:
df_teste_v2.isna().sum()

card_id              0
feature_1_1          0
feature_1_2          0
feature_1_3          0
feature_1_4          0
feature_1_5          0
feature_2_1          0
feature_2_2          0
feature_2_3          0
feature_3_1_0        0
oldest_Activation    0
latest_Activation    0
dtype: int64

In [176]:
df_treino_v2.to_csv('df_treino.csv', index=False)
df_teste_v2.to_csv('df_teste.csv', index=False)

In [190]:
# Executar último merge
treino = df_treino_v2.merge(df_historico_e_caract, on='card_id', how='inner')
teste  = df_teste_v2.merge(df_historico_e_caract,  on='card_id', how='inner')

In [182]:
# Substituir os valores NA por zero
treino = df_treino.fillna(0, axis = 0)
teste  = df_teste.fillna(0, axis  = 0)

In [ ]:
# Salvar o resultado até aqui obtido em arquivo CSV:
treino.to_csv('df_treino_ETL.csv')
teste.to_csv('df_teste_ETL.csv')

In [ ]:
# Retomando o trabalho:
# treino  = pd.read_csv('df_treino_ETL.csv')
# teste   = pd.read_csv('df_teste_ETL.csv')